In [1]:
import pandas as pd
import numpy as np

from sklearn.externals import joblib
import _pickle as pickle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier as RFC

from sklearn.utils import resample

dev_loc = r'/axp/buanalytics/csswcpfwt/dev/'

In [2]:
#pull the processed data
df_data = pd.read_csv(dev_loc+"/processed_data.csv")

#dropna, if missed in preprocess
df_data = df_data.dropna(how='any')

# for testing I have sampled, but in real-time it's not needed.
df_data = resample(df_data, replace=False, n_samples=1000, random_state=123)

df_data.head(1)

,featr_nm,featr_desc,story_nm,story_ds,PFWT
9325,ability book loan fraud wfis,ability book fraudulent merchant finance pre a...,uat test support wfis feed mf wct,mf wct fraud officer want detail fraudulent ac...,New Application Development / New App Dev Testing


In [3]:
# get x, y

i=0
for s in df_data.columns:
    if s == 'PFWT': continue
    if i==0:
        df_data["combined"] = df_data[s]
    else:
        df_data["combined"] = df_data["combined"] + df_data[s]
    i=i+1
    
#print(df_data.head(1))

x = df_data["combined"]
y = df_data['PFWT']
print(x.head(1))

9325    ability book loan fraud wfisability book fraud...
Name: combined, dtype: object


In [4]:

filename = dev_loc+'/pfwt_model.sav'
feature_file = dev_loc+"/pfwt_vectors.pkl"

loaded_vec = TfidfVectorizer(decode_error="replace", vocabulary=pickle.load(open(feature_file, "rb")))
transformer = TfidfTransformer()
x_vec = transformer.fit_transform(loaded_vec.fit_transform(x))

loaded_model = joblib.load(filename)
test_predictions = loaded_model.predict(x_vec)
test_accuracy = accuracy_score(test_predictions, y); print("testing accuracy: ", test_accuracy); print("")
print(classification_report(y, test_predictions)); print("")

testing accuracy:  0.909

                                                   precision    recall  f1-score   support

New Application Development / New App Dev Testing       0.90      0.95      0.93       515
     Other Non-Application Development activities       0.88      0.87      0.88       303
                     Research & Development (R&D)       0.99      0.85      0.91       182

                                        micro avg       0.91      0.91      0.91      1000
                                        macro avg       0.92      0.89      0.91      1000
                                     weighted avg       0.91      0.91      0.91      1000


